In [ ]:
from google.colab import drive
import pandas as pd
import sklearn
import io
drive.mount('/content/drive', force_remount = True)
path = "/content/drive/MyDrive/winequalityN.csv"
df = pd.read_csv(path)
df = df.dropna()
# df_filtered = df[df['alcohol'] <= 100]
print(df)

Mounted at /content/drive
       type  fixed acidity  volatile acidity  citric acid  residual sugar  \
0     white            7.0             0.270         0.36            20.7   
1     white            6.3             0.300         0.34             1.6   
2     white            8.1             0.280         0.40             6.9   
3     white            7.2             0.230         0.32             8.5   
4     white            7.2             0.230         0.32             8.5   
...     ...            ...               ...          ...             ...   
6491    red            6.8             0.620         0.08             1.9   
6492    red            6.2             0.600         0.08             2.0   
6494    red            6.3             0.510         0.13             2.3   
6495    red            5.9             0.645         0.12             2.0   
6496    red            6.0             0.310         0.47             3.6   

      chlorides  free sulfur dioxide  total sulfu

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

encoder = LabelEncoder()
scaler = MinMaxScaler()
df['type'] = encoder.fit_transform(df['type'])

scaled_data = scaler.fit_transform(df[[
    'type', 'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
    'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'
]])
# scaled_data = scaled_data.columns.tolist()
# scaled_data = scaler.fit_transform(df[[
#     'type', 'fixed acidity', 'volatile acidity', 'residual sugar',
#     'free sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'
# ]])

# x = np.concatenate([scaled_data, df[['type']]], axis=1)
x = scaled_data
y = df.quality
# x = df.drop('total sulfur dioxide', axis=1)

In [ ]:
!pip install pytorch-tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 50.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 whi

In [ ]:
from imblearn.over_sampling import SMOTE
# smote = SMOTE(k_neighbors = 2)
smote = SMOTE(k_neighbors = 4)
# from pytorch_tabnet.augmentations import RegressionSMOTE
# aug = RegressionSMOTE(p=0.2)

# Apply SMOTE to generate synthetic samples
x_smote, y_smote = smote.fit_resample(x, y)
y_smote = y_smote.values.reshape(-1, 1)

In [ ]:
from sklearn.model_selection import train_test_split
y = y.values.reshape(-1, 1)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size = 0.2, random_state = 42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size = 0.5, random_state = 42)

In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch

# # Preprocess the data
# target = 'quality'
# features = df.columns.tolist()
# features.remove(target)

# # Encode categorical features
# le = LabelEncoder()
# df['type'] = le.fit_transform(df['type'])

# # Split the dataset into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

# Create the TabNet classifier
# clf = TabNetClassifier()
clf = TabNetRegressor()
# clf = TabNetClassifier(n_d=64, n_a=64, n_steps=5, gamma=1.5, n_independent=2, n_shared=2,
#                        lambda_sparse=1e-3, optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.01),
#                        scheduler_fn=torch.optim.lr_scheduler.ExponentialLR, scheduler_params=dict(gamma=0.9),
#                        mask_type='entmax', device_name='auto', verbose=1, max_epochs=200)
# # Train the classifier
# clf.fit(x_train, y_train, eval_set=[(x_test, y_test)], max_epochs = 200, patience = 25)
# # Make predictions
# y_pred = clf.predict(x_test)

# # Evaluate the model
# accuracy = (y_pred == y_test).mean()
# print("Accuracy:", accuracy)

# Train the model
clf.fit(
    x_train, y_train,
    eval_set = [(x_val, y_val)],
    eval_name = ['val'],
    max_epochs = 200,
    patience = 25,
    # batch_size=1024,
    # virtual_batch_size=128,
    # num_workers=0,
    # drop_last=False,
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 28.255  | val_mse: 20.63455|  0:00:03s
epoch 1  | loss: 12.95104| val_mse: 10.53852|  0:00:04s
epoch 2  | loss: 4.48609 | val_mse: 3.31842 |  0:00:04s
epoch 3  | loss: 1.66654 | val_mse: 1.45882 |  0:00:05s
epoch 4  | loss: 1.14615 | val_mse: 1.21305 |  0:00:05s
epoch 5  | loss: 0.80541 | val_mse: 0.88468 |  0:00:06s
epoch 6  | loss: 0.72089 | val_mse: 0.77741 |  0:00:07s
epoch 7  | loss: 0.64833 | val_mse: 0.74908 |  0:00:07s
epoch 8  | loss: 0.62128 | val_mse: 0.79549 |  0:00:08s
epoch 9  | loss: 0.61262 | val_mse: 0.80631 |  0:00:08s
epoch 10 | loss: 0.59581 | val_mse: 0.79606 |  0:00:09s
epoch 11 | loss: 0.58629 | val_mse: 0.77289 |  0:00:09s
epoch 12 | loss: 0.56761 | val_mse: 0.7641  |  0:00:10s
epoch 13 | loss: 0.57202 | val_mse: 0.8081  |  0:00:11s
epoch 14 | loss: 0.56177 | val_mse: 0.77233 |  0:00:12s
epoch 15 | loss: 0.55399 | val_mse: 0.74975 |  0:00:13s
epoch 16 | loss: 0.55551 | val_mse: 0.75397 |  0:00:14s
epoch 17 | loss: 0.54595 | val_mse: 0.75142 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

# Make predictions on the test set
y_pred = clf.predict(x_test)

print("Model Before SMOTE")

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculate R-squared (R2) score
r2 = r2_score(y_test, y_pred)
print("R-squared (R2) score:", r2)

# Calculate Mean Absolute Percentage Error (MAPE)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MAPE):", mape)

# Calculate Root Mean Squared Error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared = False)
print("Root Mean Squared Error (RMSE):", rmse)

Model Before SMOTE
Mean Squared Error: 0.4675672890509228
R-squared (R2) score: 0.3821370049361461
Mean Absolute Percentage Error (MAPE): 0.09526836987739194
Root Mean Squared Error (RMSE): 0.6837889214157559


In [ ]:
from sklearn.model_selection import train_test_split
# x_train_smote, x_test_smote, y_train_smote, y_test_smote = train_test_split(x_smote, y_smote, test_size = 0.2, random_state = 42)
x_train_smote, x_temp_smote, y_train_smote, y_temp_smote = train_test_split(x_smote, y_smote, test_size = 0.2, random_state = 42)
x_val_smote, x_test_smote, y_val_smote, y_test_smote = train_test_split(x_temp_smote, y_temp_smote, test_size = 0.5, random_state = 42)

In [ ]:
# # Preprocess the data
# target = 'quality'
# features = df.columns.tolist()
# features.remove(target)

# # Encode categorical features
# le = LabelEncoder()
# df['type'] = le.fit_transform(df['type'])

# # Split the dataset into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

# Create the TabNet classifier
# clf = TabNetClassifier()
clf_smote = TabNetRegressor()
# clf = TabNetClassifier(n_d=64, n_a=64, n_steps=5, gamma=1.5, n_independent=2, n_shared=2,
#                        lambda_sparse=1e-3, optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.01),
#                        scheduler_fn=torch.optim.lr_scheduler.ExponentialLR, scheduler_params=dict(gamma=0.9),
#                        mask_type='entmax', device_name='auto', verbose=1, max_epochs=200)
# # Train the classifier
# clf.fit(x_train, y_train, eval_set=[(x_test, y_test)], max_epochs = 200, patience = 25)
# # Make predictions
# y_pred = clf.predict(x_test)

# # Evaluate the model
# accuracy = (y_pred == y_test).mean()
# print("Accuracy:", accuracy)

# Train the model
clf_smote.fit(
    x_train_smote, y_train_smote,
    eval_set = [(x_val_smote, y_val_smote)],
    eval_name = ['val'],
    max_epochs = 200,
    patience = 25,
    # batch_size=1024,
    # virtual_batch_size=128,
    # num_workers=0,
    # drop_last=False,
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 18.06952| val_mse: 3.1463  |  0:00:01s
epoch 1  | loss: 2.8476  | val_mse: 5.74339 |  0:00:02s
epoch 2  | loss: 1.76524 | val_mse: 5.6264  |  0:00:03s
epoch 3  | loss: 1.50208 | val_mse: 4.11239 |  0:00:04s
epoch 4  | loss: 1.33873 | val_mse: 3.44164 |  0:00:05s
epoch 5  | loss: 1.20921 | val_mse: 2.75318 |  0:00:06s
epoch 6  | loss: 1.1072  | val_mse: 2.79329 |  0:00:06s
epoch 7  | loss: 1.08819 | val_mse: 2.80986 |  0:00:07s
epoch 8  | loss: 0.99927 | val_mse: 2.84678 |  0:00:08s
epoch 9  | loss: 0.95905 | val_mse: 2.55133 |  0:00:10s
epoch 10 | loss: 0.96355 | val_mse: 2.41002 |  0:00:12s
epoch 11 | loss: 0.86405 | val_mse: 2.7364  |  0:00:13s
epoch 12 | loss: 0.88189 | val_mse: 2.41681 |  0:00:16s
epoch 13 | loss: 0.83165 | val_mse: 2.3573  |  0:00:19s
epoch 14 | loss: 0.82843 | val_mse: 2.38685 |  0:00:21s
epoch 15 | loss: 0.80379 | val_mse: 2.4198  |  0:00:22s
epoch 16 | loss: 0.79007 | val_mse: 2.27987 |  0:00:24s
epoch 17 | loss: 0.79346 | val_mse: 2.04556 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

# Make predictions on the test set
y_pred_smote = clf_smote.predict(x_test_smote)

print("Model After SMOTE")

# Calculate the mean squared error
mse_smote = mean_squared_error(y_test_smote, y_pred_smote)
print("Mean Squared Error:", mse_smote)

# Calculate R-squared (R2) score
r2_smote = r2_score(y_test_smote, y_pred_smote)
print("R-squared (R2) score:", r2_smote)

# Calculate Mean Absolute Percentage Error (MAPE)
mape_smote = mean_absolute_percentage_error(y_test_smote, y_pred_smote)
print("Mean Absolute Percentage Error (MAPE):", mape_smote)

# Calculate Root Mean Squared Error (RMSE)
rmse_smote = mean_squared_error(y_test_smote, y_pred_smote, squared = False)
print("Root Mean Squared Error (RMSE):", rmse_smote)

Model After SMOTE
Mean Squared Error: 0.2851647729613224
R-squared (R2) score: 0.9300739452128879
Mean Absolute Percentage Error (MAPE): 0.06786502183544806
Root Mean Squared Error (RMSE): 0.5340082143200818


In [ ]:
# from pytorch_tabnet.pretraining import TabNetPretrainer
# unsupervised_model = TabNetPretrainer()
# reconstructed_X, embedded_X = unsupervised_model.predict(x)
# assert(reconstructed_X.shape==embedded_X.shape)

# unsupervised_model_no_preproc.save_model('./test_pretrain2')
# loaded_pretrain = TabNetPretrainer()
# loaded_pretrain.load_model('./test_pretrain2.zip')